### Measure bounding box sizes

In this notebook, we'll extract a bounding box and calculate the size in centimeters and draw it onto the image.

- Since the image shapes aren't consistent and the pixel spacings are different, I figured I'd want to know the actual sizes of the BB's when comparing them together.
- We can use DICOM pixel spacing to map pixel lengths to a real world measurements (millimeters in this case).
- Typically we'd use the Pythagorean Theorem to calculate a Euclidean distance between two points. In this case, the BB's are square and their lines fall in the X and Y planes. This makes it easier to calculate their distances.
- The basic formula is -> **mm = pixel_distance * pixel_spacing**

There are various Pixel Spacing tags that can exist. Most commonly in X-Ray, we refer to the ImagerPixelSpacing tag (x0018,x1164) for measurements. This is the calibrated amount that each pixel on the receptor is multiplied by to give a real world measurement.

- Pixel Spacing tags have two values seperated with a slash character .. with one value for the X plane and one for the Y plane.
- These values are almost always the same, but they do not have to be. Some systems create images with non-square pixels (yes .. I know that seems odd, but pixels can be non-square!).
- DICOM Pixel Spacing Reference: http://dicom.nema.org/dicom/2013/output/chtml/part03/sect_10.7.html

Of the 6443 images in the train set, 5936 images have the ImagerPixelSpacing tag present.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pydicom
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join
import json
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
# Load the image data
base_path = "/kaggle/input/siim-covid19-detection/"
images_df = pd.read_csv(os.path.join(base_path,"train_image_level.csv"))

In [ ]:
# Function to get the first file in a study directory by study_id
def get_image_by_study_id(study_id):
    study_path = base_path + "train/" + study_id + "/"
    images = []
    for subdir, dirs, files in os.walk(study_path):
        for file in files:
            images.append(os.path.join(subdir, file))
            
    return images

In [ ]:
# Function to get the BB data from the images DF
def get_boxes(image_id):
    image = image_id.replace('.dcm','_image')
    ti = images_df[images_df['id'] == image]
    bx = [[],[]]
    bx[0] = [0,0,0,0,""]
    bx[1] = [0,0,0,0,""]
    
    if str(ti['boxes'].values[0]) != "nan":
        box = str(ti['boxes'].values[0]).replace("'","\"")
        boxes = json.loads(box)
        lab = ti['label'].values[0].split(" ")
        i = 0
        for b in boxes:
            bx[i] = [str(b['x']), str(b['y']), str(b['width']),str(b['height']),str(lab[0])]
            i = i+1
    return bx

In [ ]:
# Pick a random study in the train set and get image(s) from it
study_id = "00086460a852"
images = get_image_by_study_id(study_id)

In [ ]:
line_color = 'red'
text_color = 'yellow'
plt.figure(figsize=(10,10))

# Iterate through the file list
for img_file in images:
    
    # Load a file
    img = pydicom.dcmread(img_file)
    pixels = img.pixel_array
    out = apply_voi_lut(pixels, img, index=0)
    
    # Get the ImagerPixelSpacing tag
    if (0x0018,0x1164) in img:
        pixel_spacing = img[0x0018,0x1164]
        print("ImagerPixelSpacing: " + str(pixel_spacing[0]) + " / " + str(pixel_spacing[1]))
         
    # Invert MONOCHROME1
    cmap = "gray"
    if (img.PhotometricInterpretation == "MONOCHROME1"):
        cmap = "gray_r"

    # Grab the bounding boxes
    boxes = get_boxes(str(os.path.basename(img_file)))

    # Iterate through the bounding boxes and draw the lines
    if boxes:
        for i in boxes:
            distance_x = 0
            distance_y = 0
            i[0] = float(i[0])
            i[1] = float(i[1])
            i[2] = float(i[2])
            i[3] = float(i[3])

            # Top line
            x = [i[0], i[0] + i[2]]
            y = [i[1], i[1]]
            plt.plot(x,y, color=line_color, linewidth=1)
            plt.text(x[0], y[0]-10, str(i[4]), fontsize="large", color=text_color)
            
            # Bottom line
            x = [i[0], i[0] + i[2]]
            y = [i[1]+ i[3], i[1] + i[3]]
            distance_x = round((x[1] - x[0]) * pixel_spacing[0] / 10, 2)
            plt.plot(x, y, color=line_color, linewidth=1)
            plt.text(x[0] + 50, y[0] - 10, str(distance_x) + "cm", fontsize="large", color=text_color)

            # Left line
            x = [i[0], i[0]]
            y = [i[1], i[1] + i[3]]
            distance_y = round((y[1] - y[0]) * pixel_spacing[0] / 10, 2)
            plt.plot(x, y, color=line_color, linewidth=1)
            plt.text(x[0], y[0]+ ((y[1] - y[0]) / 2), str(distance_y) + "cm", fontsize="large", color=text_color)

            # Right line
            x = [i[0] + i[2], i[0] + i[2]]
            y = [i[1], i[1] + i[3]]
            plt.plot(x, y, color=line_color, linewidth=1)

    
    plt.imshow(out,cmap=cmap)
    plt.grid(False)
    plt.show()

- This could be useful for extracting ROI from the BB's and scaling them for inference .. or something.
- Likewise, other measurements on segmented masks could be useful .. possibly the Cardio-Thoracic Ratio .. or the area of a lung field.
- Any two points on an x-ray can be measured this way. Just be sure to use Pythagorean Theorem on non-orthogonal lines and watch out for non-square pixels.